# 3

This homework is designed to help you get more comfortable with two- and three-dimensional linear transformations through visualization. Furthermore, we will consider the Cholesky decomposition for symmetric positive definite linear systems.
Let's get initialization out of the way, and then proceed.

In [ ]:
import numpy as np
import scipy.linalg

import matplotlib.pyplot as plt         # plotting
import matplotlib.collections as clt    # plotting
from mpl_toolkits.mplot3d import Axes3D # 3D plotting

%matplotlib inline

from IPython.core.display import HTML
HTML("""<style>.output_png { display: table-cell; text-align: center; vertical-align: middle; }</style>""");

- - -

<div class="alert alert-info">

### Visualizing Transforms
</div>

A simple way to visualize linear transforms over two-dimensional or three-dimensional spaces is to consider their effect on a known geometry, e.g. a square, cube, circle, or sphere. Furthermore, depicting the images of basis vectors can also be helpful.

<div class="alert alert-success">

**Task 1:** Modify the function `visualize_transform_2d` below such that it draws the transformed rectangle and basis vectors corresponding to the matrix argument `M`.
</div>

In [ ]:
'''
Clearing dimension questions https://stackoverflow.com/questions/65034697/transform-rectangle-with-transformation-matrix-python
Lines and M could have different dimensions. Thankfully not a Problem here.
'''
def visualize_transform_2d( ax, M ):
    """
    Visualize the effect of the linear transform described by M on the plot axes ax.
    """ 

    # initial rectangle and basis for 2D plot.
    lines = np.array([[-1, 1, 1, -1, -1], [-1, -1, 1, 1, -1]])
    basis = np.array([[1, 0], [0, 1]])
    
    # transformation.
    trans_lines = M @ lines # @ Matrix multiplicator.
    trans_basis = M @ basis # represented by arrows.

    
    ax.plot( trans_lines[0,:], trans_lines[1,:], '-', color="gray" )
    ax.arrow( 0, 0, trans_basis[0][0], trans_basis[1][0], color='red', width=.04, length_includes_head=True )
    ax.arrow( 0, 0, trans_basis[0][1], trans_basis[1][1], color='gold', width=.04, length_includes_head=True )

    # Setting limits for x and y axes.
    ax.set_xlim( -5, 5 )
    ax.set_ylim( -5, 5 )
    ax.grid()
    
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(8,4)) # width 8, high 4 inches. (1,2,..) create grid layout of subplots with 1 row and 2 columns.

# visualize untransformed geometry
I = np.identity(2)   #np.array([[1, 0], [0, 1]])
visualize_transform_2d( ax0, I )

# visualize transformed geometry. Output because of -1 difference to identity(2).
M = np.array([[1, -1], [0, 1]])
visualize_transform_2d( ax1, M )

# Visualizations of both look the same.

<div class="alert alert-success">
    
**Task 2**: Visualize the following types of transforms using the above method:
1. A reflection on the $x$-axis (mirroring with respect to $y$ axis).
2. A *shear transform*, which is given by matrices of the form $S_a := \begin{pmatrix} 1 & a \\ 0 & 1 \end{pmatrix}$ for $a > 0$.
3. A clockwise rotation around the origin with angle $\alpha = 45^\circ$.
</div>

In [ ]:
reflection = np.array([[1, 0], [0, -1]]) # Struggels with [[1, 0], [0, 1]]

# Test.
fig, (ax0) = plt.subplots(1,  figsize=(2,2))
reflec = visualize_transform_2d( ax0, reflection)

In [ ]:
shear = np.array([[1, -1], [0, 1]])

# Test.
fig, (ax0) = plt.subplots(1,  figsize=(2,2))
shee = visualize_transform_2d( ax0, shear)

In [ ]:
# clockwise rotation.
a = np.radians(-45)
rotation = np.array([[np.cos(a), -np.sin(a)], [np.sin(a), np.cos(a)]])

# Test.
fig, (ax0) = plt.subplots(1,  figsize=(2,2))
rotat = visualize_transform_2d( ax0, rotation)

<div class="alert alert-success">

**Task 3**: Use `visualize_transform_2d` to illustrate that matrix multiplication is -- in general -- not commutative, i.e. 

$$A\cdot B \neq B\cdot A.$$ 
for two $2\times 2$-matrices $A$, $B$. To show this, find two matrices $A$ and $B$ that do not commute, and visualize the respective transforms. You can choose $A$ and $B$ from the transforms in the previous task.
<div>

In [ ]:
"""
show that A*B != B*A by using "visualize_transform_2d".
Our A is reflection and our B is shear. We are going to continue with giving the results:
"""
ab = reflection @ shear # Mamma mia, solved with ABBA :D
ba = shear @ reflection
# Create subplots for the visualizations
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(8, 4))

# Visualize shear transformation
visualize_transform_2d(ax0, ab)
ax0.set_title('A * B')

# Visualize reflection transformation
visualize_transform_2d(ax1,  ba)
ax1.set_title('B * A')

plt.tight_layout()
plt.show()
  
print("AB:\n", ab)
print("BA:\n", ba)

- - -

Visualizing 3D transforms is more challenging, but can be done in the same way in principle by drawing transformed 3D geometry. Here, we'll use a sphere instead of a rectangle. The sphere points are generated using [polar coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system) and drawn using matplotlib's [`surface`](https://matplotlib.org/3.1.1/gallery/mplot3d/surface3d.html) function.

Unfortunately, matplotlib does not have nice three-dimensional arrows, so let's stick to lines. To be able to see the basis vector inside the surface, we set the latters to be 90% transparent (the `alpha` parameter indicates opacity, i.e. the surface should be 10% opaque). 

<div class="alert alert-success">

**Task 4:** Modify the function `visualize_transform_3d` below such that it draws the transformed rectangle and basis vectors corresponding to the matrix argument `M`.
</div>

In [ ]:
def visualize_transform_3d( ax, M ):
    """Visualize the effect of the linear transform described by M on the plot axes ax."""

    # create sphere grid
    u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
    surf = np.stack([np.cos(u)*np.sin(v),np.sin(u)*np.sin(v),np.cos(v)], axis=0).reshape(3,2500)
    
    # create basis vectors for arrows
    basis = np.identity(3)
    
    # Transform surf and basis using M. Doing the same as above.
    trans_surf = M @ surf
    trans_basis = M @ basis
    
    ax.plot_surface( *(trans_surf.reshape(3,50,50)), color="gray", alpha=0.1 )

    for i, c in zip([0, 1, 2], ['red', 'gold', 'lightblue']):
        ax.plot( [0, trans_basis[0][i]], [0, trans_basis[1][i]], [0, trans_basis[2][i]], '-o', color=c )

    ax.set_xlim( -2, 2 )
    ax.set_ylim( -2, 2 )
    ax.set_zlim( -2, 2 )
    
    
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12,6), subplot_kw={'projection': '3d', 'elev': 45})

# visualize untransformed geometry
I = np.identity(3)   
visualize_transform_3d( ax0, I )

# visualize transformed geometry
M = np.array([[1, -1, 0], [0, 1, 0], [0, 0, 2]]) # Points match the right picture.
visualize_transform_3d( ax1, M )

Again, it is not too difficult to construct the matrix representation of particular transforms.

<div class="alert alert-success">

**Task 5:** Construct and visualize the following kinds of 3D linear transformations as matrices:

1. A reflection about the origin of the $xy$-plane (leaving the $z$-coordinate unchanged).
2. A rotation of angle $\alpha$ around the $x$-axis (i.e. $x$-coordinates remain unchanged).

Note: For both, consider a $2\times 2$ matrix of the same type for the relevant plane and then fill in an identity column for the unchanged coordinate.
</div>

In [ ]:
"""
A reflection about the origin of the xy-plane (leaving the 
z-coordinate unchanged).
Imagine putting a mirror on the z-plane and having the x-plane on the same interface. 
"""
reflection = np.array([
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, 1] 
])


In [ ]:
"""
A rotation of angle alpha around the x-axis 
(i.e. x-coordinates remain unchanged).
"""
# Radiant in Grad umwandeln mit:  https://www.mathway.com/de/Trigonometry .
alpha = np.pi / 4  # 45 degrees.
rotation = np.array([
    [1, 0, 0], # x-Achses keeps unchanged.
    [0, np.cos(alpha), -np.sin(alpha)], # Rotation in the y-z plane. A lot of eperiments with the trigonometric functions.
    [0, np.sin(alpha), np.cos(alpha)]   
])

# Testing like in Cell 16.
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d', 'elev': 45})

visualize_transform_3d(ax0, reflection)
ax0.set_title('reflection about the xy-plane, leaving z-coordinate unchanged.')

visualize_transform_3d(ax1, rotation)
ax1.set_title(f'Of angle alpha around x-axis, x-coordinates unchanged.')

plt.show()


- - -

<div class="alert alert-info">

### Cholesky Decomposition
</div>

As discussed in the course, a linear system with a symmetric positive definite system matrix $A\in\mathbb{R}^{n\times n}$ can be solved using the Cholesky decomposition, which we will try out in the following.

First, let's define a function to give us a random $n\times n$ system matrix $A$ and RHS $b$.

In [ ]:
def random_spd_linear_system( n ):

    # generate a random RHS
    b = 2.0 * np.random.rand(n, 1) - 1.0

    # generate a random matrix and RHS
    M = 2.0 * np.random.rand(n, n) - 1.0
    A = np.matmul(M,np.transpose(M))
#     A = np.array([[25,15,-5],[15,18,0],[-5,0,11]]) # A from the lecture example
    return A, b
    
A, b = random_spd_linear_system( 50 )

To solve the linear system $A x = b$, we can first perform *Cholesky decomposition* $A= L L^T$, where $L$ is lower triangular, giving two triangular systems

$$ L y = b \qquad\text{and}\qquad L^T x = y ,$$ 

whose solution $x$ is the solution of $Ax = b$.

<div class="alert alert-success">
    
**Task 6:** Complete the function `solve_linear_sytem_cholesky` below to return the solution of $Ax = b$.

Steps:
1. Compute Cholesky decomposition of $A = L L^T$
2. Solve $Ly = b$ using forward substitution.
3. Solve $L^Tx = y$ using backward substitution.
4. Return $x$ as the solution.

Hint: You may use `fwd_subs` and `bwd_subs` from the second Homework. You can check the correctness of your implementation by evaluating the residual vector $r = b-Ax$. 
</div>

In [ ]:
"""this is from Homework 2"""
def bwd_subs( U, y ):   # from homework 02
    n = len(U)
    # To ensure that the matrix is constructed like seen in the output below this cell.
    sol = np.empty_like(y, dtype=float)
    # range(start, until, steps) iterates over.
    for i in range(n-1, -1, -1):
        sol[i] = (y[i] - np.dot(U[i,i+1:n], sol[i+1:n]))/U[i,i]# sol[i] = (y[i] - np.dot(U[i, i+1:], sol[i+1:, i])) / U[i, i]
    return sol

"""this is from Homework 2"""
def fwd_subs( L, b ):   # from homework 02
    """Solve the linear system Ly = b with lower triangular matrix L by forward substitution."""
    y = np.empty_like(b) 
    n = len(L) #we get the length of L
    
    for i in range(n): 
        y[i] = (b[i] - (L[i, :i])@(y[:i])) / L[i, i] # works like bwd_subs, but multiplies the matrices from their first columns up till, but not including the i-th column.
            
    return y


def solve_linear_system_cholesky( A, b ):
    """Solve Ax = b using Cholesky method."""
    ls = np.zeros( A.shape )
    n = A.shape[0]
    A_prime = np.copy(A)
#     print(A_prime, "A' before \n")
    
    #step one
    for i in range (n):
        ls[i][i] = np.sqrt(A_prime[i][i])
        for j in range(i+1, n):
            ls[j][i] = A_prime[j][i] / ls[i][i]
            
        # step two
        lu = ls[:,i:i+1] # we take the i-th column of ls, one after the other
        lut = np.transpose(lu)
        A_prime -= lu@lut
        
        
#     print(A_prime, "A' at the end  \n")
    lst = np.transpose(ls) 
    y = fwd_subs(ls, b)
    x = bwd_subs(lst, y)
    return x


x = solve_linear_system_cholesky( A, b )
residual = b-np.matmul(A,x)
print(residual)